In [290]:
%pip install torch
import torch
import torch.nn as nn
import torch.nn.functional as F

Note: you may need to restart the kernel to use updated packages.


In [291]:
from sklearn.datasets import load_wine

In [292]:
wine = load_wine()

In [293]:
x = wine['data']
t = wine['target']

In [294]:
wine

{'data': array([[1.423e+01, 1.710e+00, 2.430e+00, ..., 1.040e+00, 3.920e+00,
         1.065e+03],
        [1.320e+01, 1.780e+00, 2.140e+00, ..., 1.050e+00, 3.400e+00,
         1.050e+03],
        [1.316e+01, 2.360e+00, 2.670e+00, ..., 1.030e+00, 3.170e+00,
         1.185e+03],
        ...,
        [1.327e+01, 4.280e+00, 2.260e+00, ..., 5.900e-01, 1.560e+00,
         8.350e+02],
        [1.317e+01, 2.590e+00, 2.370e+00, ..., 6.000e-01, 1.620e+00,
         8.400e+02],
        [1.413e+01, 4.100e+00, 2.740e+00, ..., 6.100e-01, 1.600e+00,
         5.600e+02]]),
 'target': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [300]:
x = torch.tensor(x,dtype=torch.float32)
t = torch.tensor(t,dtype=torch.int64)

In [302]:
type(x), type(t)

(torch.Tensor, torch.Tensor)

In [304]:
x.shape, t.shape

(torch.Size([178, 13]), torch.Size([178]))

In [306]:
dataset = torch.utils.data.TensorDataset(x,t)

In [308]:
dataset[0]

(tensor([1.4230e+01, 1.7100e+00, 2.4300e+00, 1.5600e+01, 1.2700e+02, 2.8000e+00,
         3.0600e+00, 2.8000e-01, 2.2900e+00, 5.6400e+00, 1.0400e+00, 3.9200e+00,
         1.0650e+03]),
 tensor(0))

In [310]:
#データのサンプル数 6:2:2
n_train = int(len(dataset)*0.6)
n_val = int(len(dataset)*0.2)
n_test = len(dataset) - n_train - n_val

In [312]:
n_train, n_val, n_test

(106, 35, 37)

In [314]:
torch.manual_seed(0)
train, val, test = torch.utils.data.random_split(dataset,[n_train,n_val,n_test])

In [316]:
train, val, test = torch.utils.data.random_split(dataset,[n_train,n_val,n_test])

In [318]:
x, t = next(iter(train_loader))

In [320]:
#バッチサイズ10
batch_size = 10
train_loader = torch.utils.data.DataLoader(train,batch_size,shuffle=True)
val_loader = torch.utils.data.DataLoader(val,batch_size)
test_loader = torch.utils.data.DataLoader(test,batch_size)

In [322]:
len(train), len(val), len(test)

(106, 35, 37)

In [324]:
#ネットワークの定義 4-4-3
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4,4)
        self.fc2 = nn.Linear(4,3)

    def forward(self,x):
        h = self.fc1(x)
        h = F.relu(h)
        h = self.fc2(h)
        return h

In [326]:
torch.manual_seed(0)

net = Net()

In [328]:
net

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [330]:
optimizer = torch.optim.SGD(net.parameters(),lr=0.01)

In [332]:
batch = next(iter(train_loader))
x, t = batch

In [334]:
y = net.forward(x)
y

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x13 and 4x4)

In [336]:
loss = F.cross_entropy(y, t)
loss

tensor(1.1522, grad_fn=<NllLossBackward0>)

In [338]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [340]:
optimizer.step()

In [342]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [344]:
net.to(device)

Net(
  (fc1): Linear(in_features=4, out_features=4, bias=True)
  (fc2): Linear(in_features=4, out_features=3, bias=True)
)

In [346]:
x = x.to(device)
t = t.to(device)

In [348]:
optimizer.zero_grad()

In [350]:
#エポック数
max_epoch = 1

In [352]:
torch.manual_seed(0)

In [354]:
net = Net().to(device)

In [356]:
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1)

In [358]:
for each in range(max_epoch):
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)

        print(f'loss: {loss}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x13 and 4x4)

In [360]:
#評価
x, t = next(iter(train_loader))
x = x.to(device)
t = t.to(device)
y = net(x)
y

RuntimeError: mat1 and mat2 shapes cannot be multiplied (10x13 and 4x4)

In [362]:
y_label = torch.argmax(y, dim = 1)
y_label

tensor([1, 0, 1, 1, 0, 0, 1, 1, 1, 1])

In [364]:
y_label == t

tensor([False,  True,  True, False, False, False, False, False,  True, False])

In [366]:
(y_label == t).sum().float()

tensor(3.)

In [368]:
accuracy = (y_label == t).sum().float() / len(t)
accuracy

tensor(0.3000)

In [370]:
torch.manual_seed(0)
net = Net().to(device)
optimizer = torch.optim.SGD(net.parameters(), lr = 0.1)

In [216]:
for each in range(max_epoch):
    for batch in train_loader:
        x, t = batch
        x = x.to(device)
        t = t.to(device)
        y = net(x)
        loss = F.cross_entropy(y, t)

        y_label = torch.argmax(y, dim = 1)
        accuracy = (y_label == t).sum().float() / len(t)
        print(f'accuracy: {accuracy:.2f}')

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

accuracy: 0.70
accuracy: 0.80
accuracy: 0.50
accuracy: 0.90
accuracy: 0.60
accuracy: 0.40
accuracy: 0.80
accuracy: 0.30
accuracy: 0.80


In [218]:
#正解率を計算
def calc_accuracy(data_loader):
    with torch.no_grad():
        total = 0
        correct = 0.0
        for batch in data_loader:
            x, t = batch
            x = x.to(device)
            t = t.to(device)
            y = net(x)

            y_label = torch.argmax(y, dim = 1)
            total += len(t)
            correct += (y_label == t).sum()
        accuracy = correct / total
    return accuracy

In [220]:
#検証データの正解率
calc_accuracy(val_loader)

tensor(0.4000)

In [222]:
#テストデータの正解率
calc_accuracy(test_loader)

tensor(0.4333)